# ezlocalai Tests and Examples

Simply choose your favorite model of choice from the models list and paste it into the `model` variable on the API calls. You can get a list of models below.

Install OpenAI and requests:

```bash
pip install openai requests python-dotenv
```

**Note, you do not need an OpenAI API Key, the API Key is your `EZLOCALAI_API_KEY` for the server if you defined one in your `.env` file.**

## Global definitions and helpers

Confirm that your `DEFAULT_MODEL` is set to the model you want to use in your `.env` file.


In [40]:
import openai
import requests
import time
import os
import re
from dotenv import load_dotenv

load_dotenv()

# Set your system message, max tokens, temperature, and top p here, or use the defaults.
SYSTEM_MESSAGE = "The assistant is acting as a creative writer. All of your text responses are transcribed to audio and sent to the user. Be concise with all responses. After the request is fulfilled, end with </s>."
DEFAULT_MAX_TOKENS = 256
DEFAULT_TEMPERATURE = 0.5
DEFAULT_TOP_P = 0.9

# ------------------- DO NOT EDIT BELOW THIS LINE IN THIS CELL ------------------- #
EZLOCALAI_SERVER = os.getenv("EZLOCALAI_SERVER", "http://localhost:8091")
EZLOCALAI_API_KEY = os.getenv("EZLOCALAI_API_KEY", "none")
DEFAULT_LLM = os.getenv("DEFAULT_LLM", "TheBloke/phi-2-dpo-GGUF")
openai.base_url = f"{EZLOCALAI_SERVER}/v1/"
openai.api_key = EZLOCALAI_API_KEY if EZLOCALAI_API_KEY else EZLOCALAI_SERVER
HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"{EZLOCALAI_API_KEY}",
    "ngrok-skip-browser-warning": "true",
}


def display_content(content):
    global EZLOCALAI_SERVER
    global HEADERS
    outputs_url = f"{EZLOCALAI_SERVER}/outputs/"
    os.makedirs("outputs", exist_ok=True)
    try:
        from IPython.display import Audio, display, Image, Video
    except:
        print(content)
        return
    if "http://localhost:8091/outputs/" in content:
        if outputs_url != "http://localhost:8091/outputs/":
            content = content.replace("http://localhost:8091/outputs/", outputs_url)
    if outputs_url in content:
        urls = re.findall(f"{re.escape(outputs_url)}[^\"' ]+", content)
        urls = urls[0].split("\n\n")
        for url in urls:
            file_name = url.split("/")[-1]
            url = f"{outputs_url}{file_name}"
            data = requests.get(url, headers=HEADERS).content
            if url.endswith(".jpg") or url.endswith(".png"):
                content = content.replace(url, "")
                display(Image(url=url))
            elif url.endswith(".mp4"):
                content = content.replace(url, "")
                display(Video(url=url, autoplay=True))
            elif url.endswith(".wav"):
                content = content.replace(url, "")
                display(Audio(url=url, autoplay=True))
    print(content)

## Language Models

Get a list of models to choose from if you don't already know what model you want to use.


In [41]:
# Wait for server to come up instead of timing out.
while True:
    try:
        models = requests.get(f"{EZLOCALAI_SERVER}/v1/models", headers=HEADERS)
        if models.status_code == 200:
            break
    except:
        pass
    time.sleep(1)
print(models.json())

['neoservicios/Phi-4-GGUF', 'mradermacher/Trendyol-LLM-7B-chat-v4.0.3-i1-GGUF', 'mradermacher/ChatWaifu_v1.3.1-i1-GGUF', 'mradermacher/snakmodel-7b-base-i1-GGUF', 'mradermacher/DeepSeek-V2-Grand-Horror-SMB-R1-Distill-Llama-3.1-Uncensored-16.5B-i1-GGUF', 'mradermacher/EVA-Instruct-SP-32B-GGUF', 'mradermacher/Fireball-Mistral-Nemo-Instruct-24B-merge-v1-GGUF', 'mradermacher/Hamanasu-7B-Base-i1-GGUF', 'neoservicios/DeepSeek-R1-Distill-Qwen-1.5B-GGUF', 'huskyhong/noname-ai-v3.0-medium-GGUF', 'mradermacher/Quasar-2.0-7B-i1-GGUF', 'mradermacher/EVA-Instruct-R1-i1-GGUF', 'j3st3r666/Llama-3.2-3B-Instruct-emails-GGUF', 'mradermacher/EVA-Instruct-R1-GGUF', 'MrDevolver/mergekit-slerp-tgznbww-Q3_K_S-GGUF', 'mradermacher/RakutenAI-7B-chat-GGUF', 'mradermacher/RakutenAI-7B-GGUF', 'mradermacher/RakutenAI-7B-chat-i1-GGUF', 'mradermacher/RakutenAI-7B-instruct-GGUF', 'mradermacher/RakutenAI-7B-instruct-i1-GGUF', 'mradermacher/RakutenAI-7B-i1-GGUF', 'mradermacher/FuseChat-Llama-3.2-1B-Instruct-i1-GGUF', '

## Voices

Any `wav` file in the `voices` directory will be available to use as a voice.


In [42]:
voices = requests.get(f"{EZLOCALAI_SERVER}/v1/audio/voices", headers=HEADERS)
print(voices.json())

{'voices': ['DukeNukem', 'StarTrekComputer1', 'default', 'Morgan_Freeman', 'HAL9000']}


## Vision Test


In [44]:
response = openai.chat.completions.create(
    model="deepseek-ai/deepseek-vl-1.3b-chat",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Describe each stage of this image."},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"https://www.visualwatermark.com/images/add-text-to-photos/add-text-to-image-3.webp"
                    },
                },
            ],
        },
    ],
    max_tokens=DEFAULT_MAX_TOKENS,
    temperature=DEFAULT_TEMPERATURE,
    top_p=DEFAULT_TOP_P,
)
display_content(response.choices[0].message.content)

Sure, let's break down each stage of the serene image:

1. **Birds Flying Over the Sunset Sky**: The primary focus of the image is a scene of birds flying over a body of water, silhouetted against a landscape of a beautiful sunset. The birds create an elegant silhouette against the vibrant sky, transitioning from deep blue at the horizon to warm golden orange hues near it. The sky's colors mirror the water, creating a harmonious blend of lively blues and rich oranges. Flanking these vibrant hues at the edges mean that the sunset is never fully set - it’s a brush of color that spreads across the entire sky and the water.

2. **Foreground Figure**: In the foreground, attention turns to a solitary figure standing on the shore, often interpreted as a person or a small bird. The figure is positioned facing the water, allowing them a clear panorama of the scene. The intensity of their gaze helps to orchestrate our attention to the birds and the sunset's beauty beyond.

3. **The Phrase "Free 

## Chat Completion

[OpenAI API Reference](https://platform.openai.com/docs/api-reference/chat)


In [45]:
# Modify this prompt to generate different outputs
prompt = "Write a short poem about Pikachu with a picture."


response = openai.chat.completions.create(
    model=DEFAULT_LLM,
    messages=[{"role": "user", "content": prompt}],
    temperature=DEFAULT_TEMPERATURE,
    max_tokens=DEFAULT_MAX_TOKENS,
    top_p=DEFAULT_TOP_P,
    stream=False,
    extra_body={"system_message": SYSTEM_MESSAGE},
)
display_content(response.choices[0].message.content)

In the forest's twilight, a flame did fight,
A tiny warrior, light in its might.
Pikachu, sprouting sparks from his palm,
Two colors dancing, for you and for all.

With twitchy tail and cheeks aglow,
It goes zapping, a naughty boy too.
Defying gravity, at the top he'd rise,
Fighting evil, with an unrelining embrace.

In pockets carried secretly,
He charged ahead, strong and free.
Legend in the wireless circuit glowing,
Pikachu, a hero with an indelible tracing.

Pikachu, merrily in his race,
Avatar of the future's advance.
Mounted, Chosen, ready, ignite,
Rise, you're the star against the night.

<svg><desc></desc><image xlink:href="https://example.com/pikachu.jpg"></image></svg>


## Completion

[OpenAI API Reference](https://platform.openai.com/docs/api-reference/completions/create)


In [46]:
# Modify this prompt to generate different outputs
prompt = "Write a haiku about the future."

completion = openai.completions.create(
    model=DEFAULT_LLM,
    prompt=prompt,
    temperature=DEFAULT_TEMPERATURE,
    max_tokens=DEFAULT_MAX_TOKENS,
    top_p=DEFAULT_TOP_P,
    n=1,
    stream=False,
    extra_body={"system_message": SYSTEM_MESSAGE},
)
display_content(completion.choices[0].text)

Future shimmers near,  
Threads of possibility weave,  
Unknown unfolds.


## Cloning Text to Speech

Any `wav` file in the `voices` directory can be used as a voice.


In [47]:
from pathlib import Path
import base64
import IPython.display as ipd

prompt = "Write a short poem about vikings with a picture."
os.makedirs("outputs", exist_ok=True)
audio_path = os.path.join(os.getcwd(), "outputs", f"test-speech.wav")
speech_file_path = Path(audio_path)
tts_response = openai.audio.speech.create(
    model="tts-1",
    voice="DukeNukem",
    input=prompt,
    extra_body={"language": "en"},
)
audio_content = base64.b64decode(tts_response.content)
speech_file_path.write_bytes(audio_content)
with open(audio_path, "wb") as audio_file:
    audio_file.write(audio_content)

ipd.Audio(speech_file_path)

## Audio to Text


In [48]:
with open(audio_path, "rb") as audio_file:
    transcription = openai.audio.transcriptions.create(model="base", file=audio_file)

print(transcription.text)

 Write a short poem about Vikings with a picture.


## Upload a Voice


In [49]:
upload_headers = HEADERS.copy()
del upload_headers["Content-Type"]
with open(audio_path, "rb") as audio_file:
    files = {"file": ("test-speech.wav", audio_file, "audio/wav")}
    data = {"voice": "Test"}
    response = requests.post(
        f"{EZLOCALAI_SERVER}/v1/audio/voices",
        files=files,
        data=data,
        headers=upload_headers,
    )
    print(response.json())

{'detail': 'Voice Test has been uploaded.'}


## Voice Completion Example


In [50]:
# We will use the audio response from a couple of cells back.
completion = openai.completions.create(
    model=DEFAULT_LLM,
    prompt=tts_response.content.decode("utf-8"),
    temperature=DEFAULT_TEMPERATURE,
    max_tokens=DEFAULT_MAX_TOKENS,
    top_p=DEFAULT_TOP_P,
    n=1,
    stream=False,
    extra_body={
        "system_message": SYSTEM_MESSAGE,
        "audio_format": "wav",
        "voice": "DukeNukem",
    },
)

response_text = completion.choices[0].text
display_content(response_text)

Vikings sail the stormy seas With horned helmets, and axe in hand Fear no frost, and plunder nations Left tales of valor and bold stand

In fjords they roamed, with ships as proof Of winter's wrath, they were masters To the strength of a wolf of the wilds So free and fierce, until time does part

A picture would capture their might, The painted beards, heads held high A melody of howls and songs, Immortals of the North's sky

Behind their codes, lies mystery, Unknown lands where they lived Yet their legacy reigns here With songs and tales long after



## Generate an Image


In [51]:
prompt = "Generate an image of a cat."
response = openai.images.generate(
    prompt=prompt,
    model="stabilityai/sdxl-turbo",
    n=1,
    size="512x512",
    response_format="url",
)
image = response.data[0].url
display_content(image)